In [ ]:
# data science
import scipy as sp
import numpy as np
import pandas as pd
# i/o
import json

In [ ]:
print(path_countries_per_region := '/home/weinold/github/pylcaio_integration_with_brightway/helper_functions/countries_per_region.json')
print(path_countries := '/home/weinold/github/pylcaio_integration_with_brightway/helper_functions/countries.json')

PRO_f dataframe:

| index | activityNameId | io_geography |
| ----- | -------------- | ------------ |
| 10 | 1 | RoW |
| 11 | 1 | CH |
| 12 | 1 | AT |
| 13 | 2 | RoW |
| 14 | 2 | DE |
| 15 | 2 | CH |
| 16 | 3 | FR |
| 17 | 3 | BE |
| 18 | 4 | RoW |
| 19 | 4 | CH |
| 20 | 4 | AT |

should look like:

| activityNameId | io_geography_list | RoW_region |
| -------------- | ----------------- | ---------- |
| 1 | RoW, CH, AT | RoW(1) |
| 2 | RoW, DE, CH | RoW(2) |
| 4 | RoW, CH, AT | RoW(1) |




In [ ]:
def identify_rows(self):
    """ Method to identify the various unique Rest of the World (RoW) regions of the LCA database

    Returns:
    --------
        The updated self.dictRoW in which unique RoW regions are identified in terms of countries they include

    """

    # contains a list of activityNameId's that are 'PRO_f.io_geography[i]=='RoW (=LCA RoW)'
    unique_activities_using_row = list(
        set( # removes duplicates
            self.PRO_f.activityNameId[ # nota bene: activityNameId is Master Data!
                [i for i in self.PRO_f.index if self.PRO_f.io_geography[i] == 'RoW']].tolist()
        )
    )

    RoW_activities = defaultdict(list)

    # how and WHERE are `geography` and `io_geography` matched?

    # list of tuples (activityNameId (=Master Data), geography)
    # where activityNameId where one of the geographis is 'RoW' and
    # where geography is a list of all regions that are associated with one activityNameId
    tupl = [  
        i for i in zip( # zip creates tuple
            self.PRO_f.activityNameId.loc[
                [
                    i for i in self.PRO_f.index
                    if self.PRO_f.activityNameId[i] in unique_activities_using_row
                ]
            ],
            self.PRO_f.io_geography.loc[
                [
                    i for i in self.PRO_f.index
                    if self.PRO_f.activityNameId[i] in unique_activities_using_row
                ]
            ]
        )
    ]

    # ok, dictionary is filled
    for activity, geography in tupl:
        RoW_activities[activity].append(geography)

    # remove 'RoW' from dict values
    RoW_activities = {activity: [geography1 for geography1 in geography if geography1 != 'RoW'] for activity, geography in RoW_activities.items()}

    # delete from RoW_activities processes which had only RoW as geography and are thus empty now
    for key in [i for i in list(RoW_activities.keys()) if RoW_activities[i] == []]:
        del RoW_activities[key]
        
    # put every element to the same level (elements that are lists are transformed to lists of lists)
    for values in list(RoW_activities.values()):
        for i in range(0, len(values)):
            if values[i] in self.countries_per_regions.keys():
                values[i] = self.countries_per_regions[values[i]]

    # for elements that are lists of lists stemming from the replacement of ['RER'] by [['AT','BE',...]],
    # add all of the together in a single list
    for keys in RoW_activities.keys():
        for item in RoW_activities[keys]:
            if isinstance(item, list):
                RoW_activities[keys] = sum_elements_list(RoW_activities[keys])
    # remove duplicates inside the elements
    for keys in list(RoW_activities.keys()):
        RoW_activities[keys] = list(set(RoW_activities[keys]))

    # why sort here? to ensure unique_RoWs does not contain duplicates that would just have a different order
    # need to sort to identify duplicates whose elements would be ordered differently and thus be treated as not duplicated
    for keys in RoW_activities.keys():
        RoW_activities[keys].sort()

    # identify the combination of countries that are NOT inside the residual of each process
    # dict where
    # key = activityNameId
    # value = list of IO countries that are NOT associated with activityNameId
    dictactrow = {}
    residual_geo_IO_to_remove = ['WA', 'WE', 'WF', 'WL', 'WM'] # exiobase 'rest of world' regions (all other are actual countries)
    for keys in RoW_activities.keys():
        dictactrow[keys] = list( # dictactrow[keys] is activityNameId
            set(self.listcountry) # countries + RoW regions IO
            - set(RoW_activities[keys]) # countries LCA associated with the activityNameId
            - set(residual_geo_IO_to_remove)) # RoW regions IO
    
    unique_RoWs = []
    for keys in dictactrow.keys():
        if dictactrow[keys] not in unique_RoWs:
            unique_RoWs.append(dictactrow[keys])

    # create name for the values of the different RoW
    listname = []
    for i in range(0, len(unique_RoWs)):
        listname.append('RoW' + '(' + str(i) + ')')
    
    # put all of that in dictRoW
    # dictRow is created empty earlier in the code
    for i in range(0, len(unique_RoWs)):
        self.dictRoW[listname[i]] = unique_RoWs[i]
    try:
        # if RoWs are empty because processes from ecoinvent are too described
        del [[k for k in self.dictRoW.keys() if len(self.dictRoW[k]) == 0][0]]
    except IndexError:
        pass

    # replace RoW list (eg. [AT, DE, ...] with RoW number (eg. RoW(1)))
    for activityNameId in dictactrow: # key = activityNameId
        for RoW_number in self.dictRoW: # key = eg. "RoW(12)"
            if dictactrow[activityNameId] == self.dictRoW[RoW_number]: # dictactrow[activityNameId] = RoW list, self.dictRoW[RoW_number]
                dictactrow[activityNameId] = RoW_number

    RoW_matrix = pd.DataFrame(
        data = list(dictactrow.values()),
        index=list(dictactrow.keys()),
        columns=['RoW_geography']
    )

    # adds RoW information to matrix
    self.PRO_f = self.PRO_f.merge(RoW_matrix, left_on='activityNameId', right_on=RoW_matrix.index, how='outer')

    self.PRO_f.index = self.PRO_f.activityId + '_' + self.PRO_f.productId
    self.PRO_f = self.PRO_f.reindex(self.processes_in_order)
    
    self.PRO_f.io_geography.update(self.PRO_f.RoW_geography[self.PRO_f.io_geography == 'RoW'])
    self.PRO_f = self.PRO_f.drop('RoW_geography', axis=1)

    # might be some RoW or empty lists left in PRO_f
    self.PRO_f.io_geography[self.PRO_f.io_geography == 'RoW'] = 'GLO'
    self.PRO_f.io_geography.loc[[i for i in self.PRO_f.index if type(self.PRO_f.io_geography[i]) == list]] = 'GLO'